In [ ]:
import dlib
print(dlib.DLIB_USE_CUDA)
## DROWSINESS DETECTION

False


In [5]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist

# Load face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Function to calculate Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    EAR = (A + B) / (2.0 * C)
    return EAR

# Define eye landmarks
LEFT_EYE = list(range(42, 48))
RIGHT_EYE = list(range(36, 42))

EAR_THRESHOLD = 0.25  # If below this threshold, eyes are considered closed
FRAME_THRESHOLD = 20   # Number of consecutive frames eyes must be closed

drowsy_counter = 0

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    for face in faces:
        landmarks = predictor(gray, face)
        landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])
        
        left_eye = landmarks[LEFT_EYE]
        right_eye = landmarks[RIGHT_EYE]
        
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        ear = (left_ear + right_ear) / 2.0
        
        if ear < EAR_THRESHOLD:
            drowsy_counter += 1
            if drowsy_counter >= FRAME_THRESHOLD:
                cv2.putText(frame, "DROWSINESS ALERT!", (50, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
        else:
            drowsy_counter = 0
        
        for point in np.concatenate((left_eye, right_eye), axis=0):
            cv2.circle(frame, tuple(point), 2, (0, 255, 0), -1)
    
    cv2.imshow("Drowsiness Detector", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [3]:
import urllib.request
import bz2
import shutil

url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
filename = "shape_predictor_68_face_landmarks.dat.bz2"

print("Downloading model file...")
urllib.request.urlretrieve(url, filename)

print("Extracting file...")
with bz2.BZ2File(filename) as fr, open("shape_predictor_68_face_landmarks.dat", "wb") as fw:
    shutil.copyfileobj(fr, fw)

print("Download and extraction complete!")


Extracting file...
Download and extraction complete!


In [4]:
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
